In [1]:
# pip install selenium

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from pprint import pprint

In [2]:
# webdriver 실행
browser = webdriver.Chrome()
browser.get("https://www.starbucks.co.kr/menu/drink_list.do")

In [3]:
# html source 추출
html = browser.page_source
# html parsing
soup = BeautifulSoup(html, 'html.parser')
# 원하는 항목 데이터 추출
products = soup.select('.product_list dd a')

pprint(products)

[<a class="goDrinkView" href="javascript:void(0)" prod="9200000004544"><img alt="씨솔트 카라멜 콜드 브루" src="https://image.istarbucks.co.kr/upload/store/skuimg/2025/05/[9200000004544]_20250521092626287.jpg"/></a>,
 <a class="goDrinkView" href="javascript:void(0)" prod="9200000002487"><img alt="나이트로 바닐라 크림" src="https://image.istarbucks.co.kr/upload/store/skuimg/2021/04/[9200000002487]_20210426091745467.jpg"/></a>,
 <a class="goDrinkView" href="javascript:void(0)" prod="9200000000479"><img alt="나이트로 콜드 브루" src="https://image.istarbucks.co.kr/upload/store/skuimg/2021/04/[9200000000479]_20210426091843897.jpg"/></a>,
 <a class="goDrinkView" href="javascript:void(0)" prod="9200000002081"><img alt="돌체 콜드 브루" src="https://image.istarbucks.co.kr/upload/store/skuimg/2021/04/[9200000002081]_20210415133656839.jpg"/></a>,
 <a class="goDrinkView" href="javascript:void(0)" prod="9200000002407"><img alt="리저브 나이트로" src="https://image.istarbucks.co.kr/upload/store/skuimg/2021/02/[9200000002407]_202102250951067

In [4]:
#타입 확인인
print(type(products))

<class 'bs4.element.ResultSet'>


In [ ]:
# 코드, 제품 이름
prod_cd = [[product['prod'], product.find('img')['alt']] for product in products]

pprint(prod_cd)

[['9200000004544', '씨솔트 카라멜 콜드 브루'],
 ['9200000002487', '나이트로 바닐라 크림'],
 ['9200000000479', '나이트로 콜드 브루'],
 ['9200000002081', '돌체 콜드 브루'],
 ['9200000002407', '리저브 나이트로'],
 ['9200000002093', '리저브 콜드 브루'],
 ['9200000005282', '막걸리향 크림 콜드 브루'],
 ['9200000004312', '민트 콜드 브루'],
 ['9200000000487', '바닐라 크림 콜드 브루'],
 ['9200000005991', '베르가못 콜드브루'],
 ['9200000006056', '스모크드 콜드 패션드'],
 ['9200000003661', '시그니처 더 블랙 콜드 브루'],
 ['9200000004750', '여수 윤슬 헤이즐넛 콜드브루'],
 ['9200000003285', '오트 콜드 브루'],
 ['9200000005616', '제주 비자림 리저브 콜드 브루'],
 ['9200000000038', '콜드 브루'],
 ['9200000004770', '콜드 브루'],
 ['9200000001636', '콜드 브루 몰트'],
 ['9200000001635', '콜드 브루 플로트'],
 ['9200000005378', '흑임자 크림 콜드 브루'],
 ['9200000006053', 'HP 코스타리카 TL 아이스 블랙&화이트'],
 ['106509', '아이스 커피'],
 ['2', '오늘의 커피'],
 ['9200000006051', '인도 MM 스모크드 카페 미스토'],
 ['9200000006164', '스타벅스 꿀 호떡 라떼'],
 ['9200000006165', '아이스 스타벅스 꿀 호떡 라떼'],
 ['30', '에스프레소 콘 파나'],
 ['25', '에스프레소 마키아또'],
 ['9200000005514', '스타벅스 1호점 카페 아메리카노'],
 ['9200000005516', '아이스 

In [7]:
import time
from bs4 import BeautifulSoup

result = []

for idx, prod in enumerate(prod_cd):
    container = {}
    cd, name = prod

    print(f"[{idx+1}/{len(prod_cd)}] ▶ {name} 수집 중...")

    try:
        browser.get(f"https://www.starbucks.co.kr/menu/drink_view.do?product_cd={cd}")
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        # 요소를 안전하게 추출
        volume_el = soup.select_one('.product_info_head #product_info01')
        cateL_el = soup.select_one('.smap .cate')

        if not volume_el or not cateL_el:
            print(f"⚠️ {name} 페이지에 필수 정보 없음 (용량/카테고리)")
            continue

        container['분류'] = cateL_el.get_text(strip=True)
        container['1회 제공량'] = volume_el.get_text(strip=True)
        container['메뉴'] = name

        # 아래 영양정보들도 마찬가지로 하나씩 체크
        def safe_select(selector):
            el = soup.select_one(selector)
            return el.get_text(strip=True) if el else ""

        container['칼로리(kcal)'] = safe_select('.product_info_content .kcal dd')
        container['당류(g)'] = safe_select('.product_info_content .sugars dd')
        container['단백질(g)'] = safe_select('.product_info_content .protein dd')
        container['나트륨(mg)'] = safe_select('.product_info_content .sodium dd')
        container['포화지방(g)'] = safe_select('.product_info_content .sat_FAT dd')
        container['카페인(mg)'] = safe_select('.product_info_content .caffeine dd')

        result.append(container)

    except Exception as e:
        print(f"🚨 {name} 오류 발생: {e}")

    time.sleep(3)


[1/176] ▶ 씨솔트 카라멜 콜드 브루 수집 중...
[2/176] ▶ 나이트로 바닐라 크림 수집 중...
[3/176] ▶ 나이트로 콜드 브루 수집 중...
[4/176] ▶ 돌체 콜드 브루 수집 중...
[5/176] ▶ 리저브 나이트로 수집 중...
[6/176] ▶ 리저브 콜드 브루 수집 중...
[7/176] ▶ 막걸리향 크림 콜드 브루 수집 중...
[8/176] ▶ 민트 콜드 브루 수집 중...
[9/176] ▶ 바닐라 크림 콜드 브루 수집 중...
[10/176] ▶ 베르가못 콜드브루 수집 중...
[11/176] ▶ 스모크드 콜드 패션드 수집 중...
[12/176] ▶ 시그니처 더 블랙 콜드 브루 수집 중...
[13/176] ▶ 여수 윤슬 헤이즐넛 콜드브루 수집 중...
[14/176] ▶ 오트 콜드 브루 수집 중...
[15/176] ▶ 제주 비자림 리저브 콜드 브루 수집 중...
[16/176] ▶ 콜드 브루 수집 중...
[17/176] ▶ 콜드 브루 수집 중...
[18/176] ▶ 콜드 브루 몰트 수집 중...
[19/176] ▶ 콜드 브루 플로트 수집 중...
[20/176] ▶ 흑임자 크림 콜드 브루 수집 중...
[21/176] ▶ HP 코스타리카 TL 아이스 블랙&화이트 수집 중...
[22/176] ▶ 아이스 커피 수집 중...
[23/176] ▶ 오늘의 커피 수집 중...
[24/176] ▶ 인도 MM 스모크드 카페 미스토 수집 중...
[25/176] ▶ 스타벅스 꿀 호떡 라떼 수집 중...
[26/176] ▶ 아이스 스타벅스 꿀 호떡 라떼 수집 중...
[27/176] ▶ 에스프레소 콘 파나 수집 중...
[28/176] ▶ 에스프레소 마키아또 수집 중...
[29/176] ▶ 스타벅스 1호점 카페 아메리카노 수집 중...
[30/176] ▶ 아이스 스타벅스 1호점 카페 아메리카노 수집 중...
[31/176] ▶ 아이스 카페 아메리카노 수집 중...
[32/176] ▶ 카페 아메리카노 수집 중...
[33/176

In [9]:
import pandas as pd
from pathlib import Path

# DataFrame으로 변환
df = pd.DataFrame(result)

base_dir = Path.cwd().parents[5]
csv_path = base_dir / 'resources' / 'data' / 'starbucks_menu_list.csv'

# csv 파일로 저장
df.to_csv(csv_path, index=False, encoding='utf-8-sig')
# 데이터 확인
df



,분류,1회 제공량,메뉴,칼로리(kcal),당류(g),단백질(g),나트륨(mg),포화지방(g),카페인(mg)
0,2025 Summer 1,Tall(톨) /355ml,씨솔트 카라멜 콜드 브루,150,18,1,110,6,130
1,콜드 브루,Tall(톨) /355ml,나이트로 바닐라 크림,80,10,1,40,2,232
2,콜드 브루,Tall(톨) /355ml,나이트로 콜드 브루,5,0,0,5,0,245
3,웹사이트 비노출 메뉴(사이렌오더 영양정보 연동),Tall(톨) /355ml,돌체 콜드 브루,220,22,6,80,6,155
4,콜드 브루,Tall(톨) /355ml,리저브 나이트로,5,0,0,0,0,190
...,...,...,...,...,...,...,...,...,...
171,스타벅스 주스(병음료),190 ml,블루베리 요거트 190ML,155,18,5,70,3.6,0
172,2025 Summer 1,Tall(톨) /355ml,자몽 망고 코코 프라푸치노,245,40,3,200,5,0
173,2025 Summer 1,Tall(톨) /355ml,더블 머스캣 블렌디드,165,31,0,25,0,0
174,2025 Summer 1,Tall(톨) /355ml,씨솔트 카라멜 콜드 브루,150,18,1,110,6,130


In [10]:
# 브라우저 닫기
browser.quit()

In [5]:
result = []
info = per_menu.select_one('.myAssignZone p').get_text()
menu_eng = per_menu.select_one('.myAssignZone h4 span').get_text()
menu_eng

'Bergamot Cold Brew'

In [ ]:
from pprint import pprint
import time

result = []

# 메뉴 코드와 이미지 URL 추출
prod_code_list = [prod['prod'] for prod in products]
prod_img_list = [a_tag.find('img')['src'] for a_tag in products]

# 상위 5개만 크롤링
for idx, code in enumerate(prod_code_list):
    print(f"[{idx+1}/{len(prod_code_list)}] 크롤링 중: {code}")
    browser.get(f"https://www.starbucks.co.kr/menu/drink_view.do?product_cd={code}")
    time.sleep(1)
    
    html = browser.page_source
    per_menu = BeautifulSoup(html, 'html.parser')
    
    # 각 요소 추출
    info_tag = per_menu.select_one('.myAssignZone p')
    menu_k_tag = per_menu.select_one('.myAssignZone h4')
    menu_e = per_menu.select_one('.myAssignZone h4 span')
    prod_img_tag = per_menu.select_one('.product_big_pic .zoomImg')
    prod_info = per_menu.select_one('.product_view_info')

    # 한글 메뉴명에서 <span> 제거
    if menu_k_tag:
        for span in menu_k_tag.find_all('span'):
            span.decompose()
        menu_k = menu_k_tag.get_text(strip=True)
    else:
        menu_k = '메뉴명 없음'

    info = info_tag.get_text(strip=True) if info_tag else '정보 없음'
    img_url = prod_img_tag.get('src') if prod_img_tag else prod_img_list[idx]  # 백업 이미지 URL 사용
    
    # 용량
    volume_tag = per_menu.select_one('#product_info01 p b')
    volume = volume_tag.get_text(strip=True) if volume_tag else '용량 정보 없음'

    # 영양 정보
    def get_nutrition_value(class_name):
        tag = per_menu.select_one(f'.{class_name} dd')
        return tag.get_text(strip=True) if tag else '정보 없음'

    kcal = get_nutrition_value('kcal')
    saturated_fat = get_nutrition_value('sat_FAT')
    protein = get_nutrition_value('protein')
    sodium = get_nutrition_value('sodium')
    sugars = get_nutrition_value('sugars')
    caffeine = get_nutrition_value('caffeine')

    # 알레르기 유발요인
    allergy_tag = per_menu.select_one('.product_factor p')
    allergy = allergy_tag.get_text(strip=True).replace('알레르기 유발요인 : ', '') if allergy_tag else '없음'

    # 결과 리스트에 딕셔너리로 저장
    result.append({
        'prod_code': code or '',
        '메뉴': menu_k or '',
        '영문명': menu_e or '',
        '설명': info or '',
        '이미지': img_url or '',
        '용량(ml)': volume or '',
        '칼로리': kcal or '',
        '포화지방(g)': saturated_fat or '',
        '단백질(g)': protein or '',
        '나트륨(mg)': sodium or '',
        '당류(g)': sugars or '',
        '카페인(mg)': caffeine or '',
        '알레르기': allergy or ''
    })

# 출력
print(result)


[1/176] 크롤링 중: 9200000004544
[2/176] 크롤링 중: 9200000002487
[3/176] 크롤링 중: 9200000000479
[4/176] 크롤링 중: 9200000002081
[5/176] 크롤링 중: 9200000002407
[6/176] 크롤링 중: 9200000002093
[7/176] 크롤링 중: 9200000005282
[8/176] 크롤링 중: 9200000004312
[9/176] 크롤링 중: 9200000000487
[10/176] 크롤링 중: 9200000005991
[11/176] 크롤링 중: 9200000006056
[12/176] 크롤링 중: 9200000003661
[13/176] 크롤링 중: 9200000004750
[14/176] 크롤링 중: 9200000003285
[15/176] 크롤링 중: 9200000005616
[16/176] 크롤링 중: 9200000000038
[17/176] 크롤링 중: 9200000004770
[18/176] 크롤링 중: 9200000001636
[19/176] 크롤링 중: 9200000001635
[20/176] 크롤링 중: 9200000005378
[21/176] 크롤링 중: 9200000006053
[22/176] 크롤링 중: 106509
[23/176] 크롤링 중: 2
[24/176] 크롤링 중: 9200000006051
[25/176] 크롤링 중: 9200000006164
[26/176] 크롤링 중: 9200000006165
[27/176] 크롤링 중: 30
[28/176] 크롤링 중: 25
[29/176] 크롤링 중: 9200000005514
[30/176] 크롤링 중: 9200000005516
[31/176] 크롤링 중: 110563
[32/176] 크롤링 중: 94
[33/176] 크롤링 중: 110582
[34/176] 크롤링 중: 126197
[35/176] 크롤링 중: 110601
[36/176] 크롤링 중: 38
[37/176] 크롤링 중: 9200

TypeError: can only concatenate str (not "NoneType") to str

In [8]:
print(len(result))

176


In [ ]:
import pandas as pd
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')

# 파일 경로에 날짜 포함
filename = f'../resources/data/starbucks-menu-{today}.csv'

# 저장
df = pd.DataFrame(result)
df.to_csv(filename, index=False, encoding='utf-8-sig')

NameError: name 'result' is not defined

In [28]:
browser.quit()

In [ ]:
import pandas as pd
from pathlib import Path

base_dir = Path.cwd().parents[5]
csv_path = base_dir / 'resources' / 'data' / 'starbucks-menu.csv'
starbucks_df = pd.read_csv(csv_path)
starbucks_df.head()

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기
0,9200000005991,베르가못 콜드브루,NaN,은은하게 퍼지는 플로럴 향과 베르가못의 화사하고 우아한 시트러스 맛이 느껴지는 부드...,https://image.istarbucks.co.kr/upload/store/sk...,NaN,125,1.7,3,45,22,90,우유
1,9200000004544,씨솔트 카라멜 콜드 브루,NaN,구름 처럼 부드러운 씨솔트 폼과 번트 카라멜의중독성 강한 단짠단짠 조합의 콜드 브루...,https://image.istarbucks.co.kr/upload/store/sk...,355ml,150,6,1,110,18,130,우유
2,9200000002487,나이트로 바닐라 크림,NaN,부드러운 목넘김의 나이트로 커피와 바닐라 크림의 매력을 한번에 느껴보세요!,https://image.istarbucks.co.kr/upload/store/sk...,355ml,80,2,1,40,10,232,우유
3,9200000000479,나이트로 콜드 브루,NaN,(나이트로 콜드 브루 매장 한정)나이트로 커피 정통의 캐스케이딩과 부드러운 콜드 크...,https://image.istarbucks.co.kr/upload/store/sk...,355ml,5,0,0,5,0,245,NaN
4,9200000002081,돌체 콜드 브루,NaN,"무더운 여름철,동남아 휴가지에서 즐기는 커피를 떠오르게 하는스타벅스 음료의 베스트 ...",https://image.istarbucks.co.kr/upload/store/sk...,355ml,220,6,6,80,22,155,우유


In [13]:
import os
print(os.getcwd())

d:\project\coffe.shop\src\main\java\com\example\coffe\shop\crawl


In [ ]:
starbucks_df[starbucks_df['칼로리리'].isna()]

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기


In [30]:
starbucks_df[starbucks_df['칼로리리'] == '-']

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기
65,9900000001604,에스프레소 플라이트,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,NaN,-,-,-,-,-,-,NaN
66,9900000001721,에스프레소 플라이트 도산,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,NaN,-,-,-,-,-,-,NaN


In [32]:
starbucks_df['칼로리리'] = starbucks_df['칼로리리'].replace('-', '0')

In [41]:
# '-'가 포함된 값을 가지는 row 전체 보기
contains_dash = starbucks_df.applymap(lambda x: '-' in str(x))

# 해당 조건에 맞는 row만 필터링
rows_with_dash = starbucks_df[contains_dash.any(axis=1)]

# 출력
rows_with_dash

C:\Users\kyjyj\AppData\Local\Temp\ipykernel_20672\1630318936.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_dash = starbucks_df.applymap(lambda x: '-' in str(x))


,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기
65,9900000001604,에스프레소 플라이트,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,NaN,0,-,-,-,-,-,NaN
66,9900000001721,에스프레소 플라이트 도산,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,NaN,0,-,-,-,-,-,NaN
122,9200000006218,MINI 라임 패션 티 with 팔삭 셔벗,NaN,"*ALL-ELECTRIC MINI ACEMAN에서 영감을 받아 탄생한, MINI와 ...",https://image.istarbucks.co.kr/upload/store/sk...,355ml,170,0,0,25,34,35,복숭아


In [44]:
starbucks_df[starbucks_df['용량(ml)'].isna()]

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기
0,9200000005991,베르가못 콜드브루,NaN,은은하게 퍼지는 플로럴 향과 베르가못의 화사하고 우아한 시트러스 맛이 느껴지는 부드...,https://image.istarbucks.co.kr/upload/store/sk...,NaN,125,1.7,3,45,22,90,우유
12,9200000004750,여수 윤슬 헤이즐넛 콜드브루,NaN,"햇빛이나 달빛에 비치어 반짝이는 잔물결이라는""윤슬""을 형상화한 헤이즐넛 콜드브루",https://image.istarbucks.co.kr/upload/store/sk...,NaN,240,9,5,80,27,53,우유
28,9200000005514,스타벅스 1호점 카페 아메리카노,NaN,풍부하고 부드러운 바디감의 별다방 블렌드 원두로 내린 에스프레소와 뜨거운 물을 섞어...,https://image.istarbucks.co.kr/upload/store/sk...,NaN,10,0,1,0,0,210,NaN
29,9200000005516,아이스 스타벅스 1호점 카페 아메리카노,NaN,풍부하고 부드러운 바디감의 별다방 블렌드 원두로 내린 에스프레소와 얼음과 물을 섞어...,https://image.istarbucks.co.kr/upload/store/sk...,NaN,10,0,1,0,0,210,NaN
40,9200000005518,스타벅스 1호점 카페 라떼,NaN,풍부하고 부드러운 바디감의 별다방 블렌드 원두로 내린 에스프레소와 신선한 우유를 스...,https://image.istarbucks.co.kr/upload/store/sk...,NaN,200,6,10,150,15,210,우유
41,9200000005522,스타벅스 1호점 크림 라떼,NaN,"""Coffee & Tea & Spice""스타벅스 첫번째 로고에서 착안한 음료로 스파...",https://image.istarbucks.co.kr/upload/store/sk...,NaN,215,12,2,25,19,210,우유
45,9200000005520,아이스 스타벅스 1호점 카페 라떼,NaN,풍부하고 부드러운 바디감의 별다방 블렌드 원두로 내린 에스프레소와 신선한 우유와 얼...,https://image.istarbucks.co.kr/upload/store/sk...,NaN,110,3.2,6,80,8,210,우유
46,9200000005523,아이스 스타벅스 1호점 크림 라떼,NaN,"""Coffee & Tea & Spice""스타벅스 첫번째 로고에서 착안한 음료로 스파...",https://image.istarbucks.co.kr/upload/store/sk...,NaN,215,12,2,25,19,210,우유
65,9900000001604,에스프레소 플라이트,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,NaN,0,-,-,-,-,-,NaN
66,9900000001721,에스프레소 플라이트 도산,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,NaN,0,-,-,-,-,-,NaN


In [48]:
starbucks_df['용량(ml)'] = starbucks_df['용량(ml)'].fillna(0)

In [49]:
starbucks_df[starbucks_df['용량(ml)'].isna()]

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기


In [ ]:
starbucks_df[starbucks_df['단백질(g)'] == '-']

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기
65,9900000001604,에스프레소 플라이트,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,0,0,-,-,-,-,-,NaN
66,9900000001721,에스프레소 플라이트 도산,NaN,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,0,0,-,-,-,-,-,NaN


In [52]:
# '-' 문자열을 0으로 변환
starbucks_df = starbucks_df.replace('-', 0)

# NaN (결측값)을 0으로 변환
starbucks_df = starbucks_df.fillna(0)

In [65]:
starbucks_df[starbucks_df['메뉴'] == '에스프레소 플라이트']

,prod_code,메뉴,영문명,설명,이미지,용량(ml),칼로리리,포화지방(g),단백질(g),나트륨(mg),당류(g),카페인(mg),알레르기
65,9900000001604,에스프레소 플라이트,0.0,스타벅스 리저브 에스프레소를 새롭고 다양하게 즐길 수 있는 에스프레소 플라이트,https://image.istarbucks.co.kr/upload/store/sk...,0,0,0,0,0,0,0,0


In [69]:
import pandas as pd

df = pd.DataFrame(starbucks_df)
base_dir = Path.cwd().parents[5]
csv_path = base_dir / 'resources' / 'data' / 'starbucks-menu-pre.csv'

# CSV 저장
df.to_csv(csv_path, index=False, encoding='utf-8-sig')